## Segmenting and Clustering Neighborhoods in Toronto

In [0]:
import pandas as pd
import folium

#### Get the csv file saved before

In [2]:
df = pd.read_csv('toronto_geo.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,0,0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,1,1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,2,2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,3,3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,4,4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


## Generate maps to visualize  neighborhoods

In [0]:
latitude = df.loc[0]['Latitude']
longitude = df.loc[0]['Longitude']
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [4]:
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto